# Multi-Dataset Segmentation Performance Visualization
## Professional IEEE-style figures for Bohn FCD (T1, FLAIR) and ISLES 2022 (DWI, FLAIR, ADC) with 5 models

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# IEEE publication style settings
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 9,
    'legend.title_fontsize': 10,
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'figure.figsize': (7, 4),
    'axes.linewidth': 0.8,
    'grid.linewidth': 0.5,
    'lines.linewidth': 2,
    'lines.markersize': 7
})

# Dataset name mapping for display
dataset_display_names = {
    'FCD': 'Bohn FCD',
    'ISLES': 'ISLES 2022'
}

# Consistent color mapping for models (matching visualization_isbi.ipynb)
model_colors = {
    'From Scratch': '#1f77b4',  # blue
    'CL': '#ff7f0e',  # orange
    'MCL': '#2ca02c',  # green
    'MAE + CL': '#d62728',  # red
    'MAE + MCL': '#9467bd'  # purple
}

# Professional color palette for modalities
modality_colors = {'T1': '#1f77b4', 'FLAIR': '#ff7f0e', 'DWI': '#2ca02c', 'ADC': '#d62728'}

# Desired model order for plotting
model_order = ['From Scratch', 'CL', 'MCL', 'MAE + CL', 'MAE + MCL']

In [ ]:
# Results from both datasets
results = {
    # FCD Dataset
    "FCD_flair_fewshot-100%_from_scratch": 0.14429,
    "FCD_flair_fewshot-100%_combined_modality": 0.16486,
    "FCD_flair_fewshot-100%_combined_regular": 0.188813,
    "FCD_flair_fewshot-100%_contrastive_modality": 0.14693,
    "FCD_flair_fewshot-100%_contrastive_regular": 0.11067,
    
    "FCD_t1_fewshot-100%_from_scratch": 0.045786,
    "FCD_t1_fewshot-100%_combined_modality": 0.1512,
    "FCD_t1_fewshot-100%_combined_regular": 0.17005,
    "FCD_t1_fewshot-100%_contrastive_modality": 0.10612,
    "FCD_t1_fewshot-100%_contrastive_regular": 0.11182,
    
    # ISLES Dataset
    "ISLES_dwi_fewshot-100%_from_scratch": 0.72504,
    "ISLES_dwi_fewshot-100%_combined_modality": 0.74233,
    "ISLES_dwi_fewshot-100%_combined_regular": 0.75502,
    "ISLES_dwi_fewshot-100%_contrastive_modality": 0.72776,
    "ISLES_dwi_fewshot-100%_contrastive_regular": 0.74996,
    
    
    "ISLES_flair_fewshot-100%_from_scratch": 0.53582,
    "ISLES_flair_fewshot-100%_combined_modality": 0.5351,
    "ISLES_flair_fewshot-100%_combined_regular": 0.56611,
    "ISLES_flair_fewshot-100%_contrastive_modality": 0.52077,
    "ISLES_flair_fewshot-100%_contrastive_regular": 0.50026,
    
    "ISLES_adc_fewshot-100%_from_scratch": 0.46088,
    "ISLES_adc_fewshot-100%_combined_modality": 0.47238,
    "ISLES_adc_fewshot-100%_combined_regular": 0.49745,
    "ISLES_adc_fewshot-100%_contrastive_modality": 0.44796,
    "ISLES_adc_fewshot-100%_contrastive_regular": 0.44921,
}

# Parse results into structured data
data_rows = []
for key, dice_score in results.items():
    parts = key.split('_')
    dataset = parts[0]  # FCD or ISLES
    modality = parts[1].upper()  # T1, FLAIR, DWI
    model = '_'.join(parts[3:])  # model name
    
    # Format model name for display
    model_display = model.replace('_', ' ').title()
    
    # Map to preferred labels
    label_mapping = {
        'Contrastive Regular': 'CL',
        'Contrastive Modality': 'MCL',
        'Combined Regular': 'MAE + CL',
        'Combined Modality': 'MAE + MCL'
    }
    model_display = label_mapping.get(model_display, model_display)
    
    data_rows.append({
        'Dataset': dataset,
        'Modality': modality,
        'Model': model_display,
        'Dice Score': dice_score
    })

df = pd.DataFrame(data_rows)
print("\n" + "="*80)
print("DATA SUMMARY")
print("="*80)
print(df.to_string(index=False))
print("\n")

## Figure 1: Side-by-Side Dataset Comparison
**Main Figure** - Faceted plot showing model performance across modalities for Bohn FCD and ISLES 2022

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4.5), sharey=False)

datasets = ['FCD', 'ISLES']

for idx, dataset in enumerate(datasets):
    ax = axes[idx]
    df_dataset = df[df['Dataset'] == dataset]
    
    # Get modalities for this dataset
    modalities = sorted(df_dataset['Modality'].unique())
    # Use the defined model order instead of sorting
    models = [m for m in model_order if m in df_dataset['Model'].unique()]
    
    x = np.arange(len(models))
    # Slightly wider bars for 3 modalities to fit numbers better
    width = 0.35 if len(modalities) == 2 else 0.28
    
    # Plot bars for each modality
    for i, modality in enumerate(modalities):
        df_mod = df_dataset[df_dataset['Modality'] == modality]
        values = []
        for model in models:
            val = df_mod[df_mod['Model'] == model]['Dice Score'].values
            values.append(val[0] if len(val) > 0 else 0)
        
        offset = width * (i - (len(modalities)-1)/2)
        bars = ax.bar(x + offset, values, width, label=modality,
                      color=modality_colors.get(modality, '#666666'), 
                      alpha=0.9, edgecolor='black', linewidth=0.5)
        
        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            if height > 0.01:  # Only show if significant
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.3f}',
                       ha='center', va='bottom', fontsize=7)
    
    ax.set_xlabel('Model')
    ax.set_ylabel('Dice Score')
    # Use display name for dataset
    dataset_display = dataset_display_names.get(dataset, dataset)
    ax.set_title(f'{dataset_display}: Model Performance Across Modalities', fontweight='bold')
    
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=45, ha='right')
    
    # Add padding to x-axis limits for better spacing
    ax.set_xlim(-0.6, len(models) - 0.4)
    
    # Legend - placed outside the plot area on the right
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=True, 
              edgecolor='black', fancybox=False, shadow=False, title='Modality')
    
    # Grid
    ax.yaxis.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Clean spines
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    
    # Set y-axis range with some padding
    max_val = df_dataset['Dice Score'].max()
    ax.set_ylim(0, max_val * 1.15)

plt.tight_layout()
plt.savefig('multi_dataset_comparison.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.savefig('multi_dataset_comparison.pdf', bbox_inches='tight',
            facecolor='white', edgecolor='none')
print("✓ Saved: multi_dataset_comparison.png/pdf")
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(7, 9), sharey=False)

datasets = ['FCD', 'ISLES']

for idx, dataset in enumerate(datasets):
    ax = axes[idx]
    df_dataset = df[df['Dataset'] == dataset]
    
    # Get modalities for this dataset
    modalities = sorted(df_dataset['Modality'].unique())
    # Use the defined model order instead of sorting
    models = [m for m in model_order if m in df_dataset['Model'].unique()]
    
    x = np.arange(len(models))
    # Slightly wider bars for 3 modalities to fit numbers better
    width = 0.35 if len(modalities) == 2 else 0.28
    
    # Plot bars for each modality
    for i, modality in enumerate(modalities):
        df_mod = df_dataset[df_dataset['Modality'] == modality]
        values = []
        for model in models:
            val = df_mod[df_mod['Model'] == model]['Dice Score'].values
            values.append(val[0] if len(val) > 0 else 0)
        
        offset = width * (i - (len(modalities)-1)/2)
        bars = ax.bar(x + offset, values, width, label=modality,
                      color=modality_colors.get(modality, '#666666'), 
                      alpha=0.9, edgecolor='black', linewidth=0.5)
        
        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            if height > 0.01:  # Only show if significant
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.3f}',
                       ha='center', va='bottom', fontsize=7)
    
    ax.set_xlabel('Model')
    ax.set_ylabel('Dice Score')
    # Use display name for dataset
    dataset_display = dataset_display_names.get(dataset, dataset)
    ax.set_title(f'{dataset_display}: Model Performance Across Modalities', fontweight='bold')
    
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=45, ha='right')
    
    # Add padding to x-axis limits for better spacing
    ax.set_xlim(-0.6, len(models) - 0.4)
    
    # Legend - placed outside the plot area on the right
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=True, 
              edgecolor='black', fancybox=False, shadow=False, title='Modality')
    
    # Grid
    ax.yaxis.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Clean spines
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    
    # Set y-axis range with some padding
    max_val = df_dataset['Dice Score'].max()
    ax.set_ylim(0, max_val * 1.15)

plt.tight_layout()
plt.savefig('multi_dataset_comparison_vertical.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.savefig('multi_dataset_comparison_vertical.pdf', bbox_inches='tight',
            facecolor='white', edgecolor='none')
print("✓ Saved: multi_dataset_comparison_vertical.png/pdf")
plt.show()

## Figure 1b: Vertical Stacked Dataset Comparison
**Alternative Layout** - Vertical stacked plot with Bohn FCD on top and ISLES 2022 below (single column format)

## Figure 2: Comprehensive Heatmap
Complete performance matrix across all datasets, modalities, and models

In [ ]:
# Create a combined dataset-modality identifier with display names
df['Dataset-Modality'] = df['Dataset'].map(dataset_display_names) + '\n' + df['Modality']

# Pivot data for heatmap
pivot_df = df.pivot_table(index='Model', columns='Dataset-Modality', 
                          values='Dice Score', aggfunc='first')

# Sort columns to group by dataset
col_order = sorted(pivot_df.columns, key=lambda x: (x.split('\n')[0], x.split('\n')[1]))
pivot_df = pivot_df[col_order]

# Reorder rows by model_order
row_order = [m for m in model_order if m in pivot_df.index]
pivot_df = pivot_df.loc[row_order]

fig, ax = plt.subplots(figsize=(8, 5))

# Create heatmap with appropriate vmax based on data
vmax = pivot_df.max().max()
im = ax.imshow(pivot_df.values, cmap='YlOrRd', aspect='auto', vmin=0, vmax=vmax)

# Set ticks
ax.set_xticks(np.arange(len(pivot_df.columns)))
ax.set_yticks(np.arange(len(pivot_df.index)))
ax.set_xticklabels(pivot_df.columns, fontsize=9)
ax.set_yticklabels(pivot_df.index)

# Rotate the tick labels
plt.setp(ax.get_xticklabels(), rotation=0, ha="center")

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Dice Score', rotation=270, labelpad=15)

# Add text annotations
for i in range(len(pivot_df.index)):
    for j in range(len(pivot_df.columns)):
        value = pivot_df.values[i, j]
        if not np.isnan(value):
            # Choose text color based on background
            text_color = 'white' if value > vmax * 0.6 else 'black'
            ax.text(j, i, f'{value:.3f}',
                   ha="center", va="center", color=text_color, 
                   fontsize=8, weight='bold')

ax.set_title('Multi-Dataset Performance Matrix: All Models and Modalities', fontweight='bold')
ax.set_xlabel('Dataset - Modality')
ax.set_ylabel('Model')

plt.tight_layout()
plt.savefig('comprehensive_heatmap.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.savefig('comprehensive_heatmap.pdf', bbox_inches='tight',
            facecolor='white', edgecolor='none')
print("✓ Saved: comprehensive_heatmap.png/pdf")
plt.show()

## Figure 3: Model Consistency Across Datasets
Shows how consistently each model performs across different datasets/modalities

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

# Get all unique dataset-modality combinations
dataset_modalities = sorted(df['Dataset-Modality'].unique())
# Use the defined model order instead of sorting
models = [m for m in model_order if m in df['Model'].unique()]

x = np.arange(len(dataset_modalities))
width = 0.15

# Plot bars for each model with consistent colors
for i, model in enumerate(models):
    df_model = df[df['Model'] == model]
    values = []
    for dm in dataset_modalities:
        val = df_model[df_model['Dataset-Modality'] == dm]['Dice Score'].values
        values.append(val[0] if len(val) > 0 else 0)
    
    offset = width * (i - 2)
    bars = ax.bar(x + offset, values, width, label=model,
                  color=model_colors.get(model, '#666666'), alpha=0.9,
                  edgecolor='black', linewidth=0.5)

ax.set_xlabel('Dataset - Modality')
ax.set_ylabel('Dice Score')
ax.set_title('Model Consistency: Performance Across All Dataset-Modality Combinations', fontweight='bold')

ax.set_xticks(x)
ax.set_xticklabels(dataset_modalities, rotation=0, ha='center')

# Legend - placed outside the plot area on the right
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=True, 
          edgecolor='black', fancybox=False, shadow=False)

# Grid
ax.yaxis.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
ax.set_axisbelow(True)

# Clean spines
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

plt.tight_layout()
plt.savefig('model_consistency.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.savefig('model_consistency.pdf', bbox_inches='tight',
            facecolor='white', edgecolor='none')
print("✓ Saved: model_consistency.png/pdf")
plt.show()

## Summary Statistics

In [6]:
print("\n" + "="*80)
print("COMPREHENSIVE SUMMARY STATISTICS")
print("="*80)

# Best per dataset
print("\n1. BEST PERFORMANCE PER DATASET:")
print("-" * 80)
for dataset in sorted(df['Dataset'].unique()):
    df_dataset = df[df['Dataset'] == dataset]
    # Exclude zero scores from scratch models
    df_dataset_nonzero = df_dataset[df_dataset['Dice Score'] > 0]
    if len(df_dataset_nonzero) > 0:
        best = df_dataset_nonzero.loc[df_dataset_nonzero['Dice Score'].idxmax()]
        print(f"   {dataset:8s}: {best['Model']:25s} with {best['Modality']:6s} (Dice = {best['Dice Score']:.4f})")

# Best per modality within each dataset
print("\n2. BEST MODEL PER MODALITY (BY DATASET):")
print("-" * 80)
for dataset in sorted(df['Dataset'].unique()):
    print(f"   {dataset}:")
    df_dataset = df[df['Dataset'] == dataset]
    for modality in sorted(df_dataset['Modality'].unique()):
        df_mod = df_dataset[df_dataset['Modality'] == modality]
        df_mod_nonzero = df_mod[df_mod['Dice Score'] > 0]
        if len(df_mod_nonzero) > 0:
            best = df_mod_nonzero.loc[df_mod_nonzero['Dice Score'].idxmax()]
            print(f"      {modality:6s}: {best['Model']:25s} (Dice = {best['Dice Score']:.4f})")

# Model consistency - average performance across all dataset-modality combinations
print("\n3. MODEL CONSISTENCY (AVERAGE ACROSS ALL COMBINATIONS, EXCLUDING ZEROS):")
print("-" * 80)
df_nonzero = df[df['Dice Score'] > 0]
model_avg = df_nonzero.groupby('Model')['Dice Score'].agg(['mean', 'std', 'min', 'max'])
model_avg = model_avg.sort_values('mean', ascending=False)
for model in model_avg.index:
    stats = model_avg.loc[model]
    print(f"   {model:25s}: Mean={stats['mean']:.4f}, Std={stats['std']:.4f}, "
          f"Range=[{stats['min']:.4f}, {stats['max']:.4f}]")

# Overall best
print("\n4. OVERALL BEST PERFORMANCE:")
print("-" * 80)
best_overall = df.loc[df['Dice Score'].idxmax()]
print(f"   {best_overall['Model']:25s} with {best_overall['Dataset']:8s} - {best_overall['Modality']:6s}")
print(f"   Dice Score: {best_overall['Dice Score']:.4f}")

# Dataset difficulty comparison
print("\n5. DATASET DIFFICULTY (AVERAGE DICE SCORES, EXCLUDING ZEROS):")
print("-" * 80)
dataset_avg = df_nonzero.groupby('Dataset')['Dice Score'].mean().sort_values(ascending=False)
for dataset in dataset_avg.index:
    print(f"   {dataset:8s}: {dataset_avg[dataset]:.4f} (easier task)" if dataset_avg[dataset] > 0.5 else f"   {dataset:8s}: {dataset_avg[dataset]:.4f} (harder task)")

# Cross-dataset modality comparison (for FLAIR which appears in both)
print("\n6. CROSS-DATASET MODALITY COMPARISON (FLAIR):")
print("-" * 80)
df_flair = df[df['Modality'] == 'FLAIR']
for dataset in sorted(df_flair['Dataset'].unique()):
    df_flair_dataset = df_flair[(df_flair['Dataset'] == dataset) & (df_flair['Dice Score'] > 0)]
    avg_score = df_flair_dataset['Dice Score'].mean()
    print(f"   {dataset:8s} FLAIR: Average Dice = {avg_score:.4f}")

print("\n" + "="*80)


COMPREHENSIVE SUMMARY STATISTICS

1. BEST PERFORMANCE PER DATASET:
--------------------------------------------------------------------------------
   FCD     : Combined Regular          with FLAIR  (Dice = 0.1888)
   ISLES   : Combined Regular          with DWI    (Dice = 0.7550)

2. BEST MODEL PER MODALITY (BY DATASET):
--------------------------------------------------------------------------------
   FCD:
      FLAIR : Combined Regular          (Dice = 0.1888)
      T1    : Combined Regular          (Dice = 0.1701)
   ISLES:
      ADC   : Combined Regular          (Dice = 0.4975)
      DWI   : Combined Regular          (Dice = 0.7550)
      FLAIR : Combined Regular          (Dice = 0.5661)

3. MODEL CONSISTENCY (AVERAGE ACROSS ALL COMBINATIONS, EXCLUDING ZEROS):
--------------------------------------------------------------------------------
   Combined Regular         : Mean=0.4355, Std=0.2521, Range=[0.1701, 0.7550]
   Combined Modality        : Mean=0.4132, Std=0.2535, Range=[0

## Output Files Generated

### Publication-Ready Figures (300 DPI):
1. **multi_dataset_comparison.png/pdf** - Side-by-side faceted comparison of Bohn FCD and ISLES 2022
2. **comprehensive_heatmap.png/pdf** - Complete performance matrix across all datasets/modalities
3. **model_consistency.png/pdf** - Model performance consistency across different conditions

### Key Features:
- **IEEE publication style** (Times New Roman, serif fonts)
- **Professional color palette** (distinct colors for each modality)
- **High resolution**: 300 DPI PNG + vector PDF for LaTeX
- **Clean design** with minimal spines and subtle grids
- **Value labels** on all bars for easy reading
- **Automatic scaling** based on data range

### Key Findings:
- **ISLES 2022 is an easier task** (Dice ~0.75) compared to Bohn FCD (Dice ~0.19)
- **Combined Regular** shows the best performance across both datasets
- **From Scratch** baseline shows significantly lower performance (as expected)
- **FLAIR** performs better than T1 on Bohn FCD dataset
- **DWI and FLAIR** show similar performance on ISLES 2022 dataset

### For Publications:
Use the **multi_dataset_comparison** as your main figure - it clearly shows:
- Model performance comparisons
- Modality differences
- Cross-dataset generalization

The **comprehensive_heatmap** is perfect for supplementary materials showing the complete results matrix.

# Multi-Checkpoint Embedding Extraction
## Extract 768-dim embeddings (before contrastive probing layer) from 4 different finetuned checkpoints